In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**first we start with importing the libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.image as mpimg
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter, MonthLocator
from matplotlib.ticker import NullFormatter
import seaborn as sns



import os
from collections import OrderedDict
from termcolor import colored
from collections import Counter, defaultdict
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px
import squarify 
import re 
import datetime as dt
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**We read the data we have for covid world vaccination, 3 datasets for 
1 population by country
2 covid world vaccination
3 global dataset**

In [ ]:
df = pd.read_csv('/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv')
df['date'] = pd.to_datetime(df['date'])

df_daily = pd.read_csv('/kaggle/input/covid19-global-dataset/worldometer_coronavirus_daily_data.csv')
df_daily['date'] = pd.to_datetime(df_daily['date'])

df_pop = pd.read_csv('/kaggle/input/population-by-country-2020/population_by_country_2020.csv')

**List for covid vaccines**

In [ ]:
vaccines = df['vaccines'].to_list()

# Remove Duplicate from vaccines list 

vaccine_list = list(OrderedDict.fromkeys(vaccines))
vaccines_text =""

for i in vaccine_list:
    vaccines_text = vaccines_text  + ' '+ i

lists = vaccines_text.replace(',', ' ')
lists.split()


vacc_list = list(OrderedDict.fromkeys(lists.split()))


#print(vacc_list)

# revise Sputnik & V to Sputnik-V 


vacc_list.remove('Sputnik')
vacc_list.remove('V')
vacc_list.append('Sputnik-V')

#print(vacc_list)

print('')
print('Current vaccines available: ', len(vacc_list))
print('')

count = 0
for i in vacc_list:
    print(count+1, i)
    count += 1

**What vaccines are used in each country?**

ANSWER:

In [ ]:
print('')
print('There are', len(list(OrderedDict.fromkeys(df['country']))), 'countries with', len(list(OrderedDict.fromkeys(df['vaccines']))), 'vaccines type groups')
print('')

In [ ]:
# To remove duplicates
vaccine_list = list(OrderedDict.fromkeys(df['vaccines']))

# conver the list to str
def covert_list_to_str(org_list, seperator= ' ' ):
    return seperator.join(org_list)

# now we display countries of each Vaccines group.

for i in vaccine_list:
    countries = df[df['vaccines'] == i]
    print('')
    print(colored(i,'blue',attrs=['bold']))
    a = list(OrderedDict.fromkeys(countries['country']))
    full_str =   covert_list_to_str(a,', ')
    print(full_str)

**Top Countries**

In [ ]:
vaccine_type = df.groupby(['country'])['total_vaccinations'].max().sort_values(ascending=False).reset_index()

#plot Top all Countries by % People vaccinated

vaccine_type.reset_index()
f,ax = plt.subplots(1,1,figsize=(25,30))

cnt = len(list(OrderedDict.fromkeys(df['country'])))
sns.barplot(y='country', x='total_vaccinations', data = vaccine_type.head(cnt))

y = vaccine_type['total_vaccinations'].head(cnt)
for i, v in enumerate(y):   
    ax.text(v+1, i+.25, str(v), fontsize=8, color='blue')

plt.title('Top Countries by total_vaccinations', weight='bold', size=15)
plt.ylabel('')
plt.xlabel('total_vaccinations')
plt.grid(axis='x')
plt.show()

In [ ]:
top_percent_people_vacc = df.groupby(['country'])['people_vaccinated_per_hundred'].max().sort_values(ascending=False).reset_index()

In [ ]:
#plot Top all Countries by % People vaccinated

top_percent_people_vacc.reset_index()
f,ax = plt.subplots(1,1,figsize=(25,40))

count = 211
sns.barplot(y='country', x='people_vaccinated_per_hundred', data = top_percent_people_vacc.head(count))

y = top_percent_people_vacc['people_vaccinated_per_hundred'].head(count)
for i, v in enumerate(y):   
    ax.text(v+1, i+.25, str(v), fontsize=8, color='blue')

plt.title('Top Countries by % People vaccinated', weight='bold', size=15)
plt.ylabel('')
plt.xlabel('%People vaccinated')
plt.grid(axis='x')
plt.show()

**Top Countries by %people fully vaccinated**

In [ ]:
top_percent_people_fvacc = df.groupby(['country'])['people_fully_vaccinated_per_hundred'].max().sort_values(ascending=False).reset_index()

In [ ]:
#plot Top all Countries by % People vaccinated

top_percent_people_fvacc.reset_index()
f,ax = plt.subplots(1,1,figsize=(25,40))

sns.barplot(y='country', x='people_fully_vaccinated_per_hundred', data = top_percent_people_fvacc.head(cnt))

y = top_percent_people_fvacc['people_fully_vaccinated_per_hundred'].head(cnt)
for i, v in enumerate(y):   
    ax.text(v+0.5, i+.25, str(v), fontsize=8, color='blue')

plt.title('Top Countries by % People Fully vaccinated', weight='bold', size=18)
plt.ylabel('')
plt.xlabel('%People Fully vaccinated')
plt.grid(axis='x')
plt.show()

**Total Vaccination Vs Population**

In [ ]:
vaccine_type = df.groupby(['country'])['total_vaccinations'].max().sort_values(ascending=False).reset_index()

fully_vaccine_countries =[]
fully_vaccine_pop=[]
people_fvacct = []


countries = list(vaccine_type['country'])
for pop in countries:
    pop1 = df_pop[df_pop['Country (or dependency)'] == pop]
    pop2 = pop1['Population (2020)'].unique()
    for number in pop2:
        fully_vaccine_pop.append(number)   
        fully_vaccine_countries.append(pop)

data = pd.DataFrame({'countries': fully_vaccine_countries,
                     'Population (2020)': fully_vaccine_pop,
                        })

countries = list(data['countries'])

for pop in countries:
    pop3 = vaccine_type[vaccine_type['country'] == pop]
    pop4 = pop3['total_vaccinations'].unique()
    
    for number in pop4:
        people_fvacct.append(number)
  
      
data2 = pd.DataFrame({'countries': fully_vaccine_countries,
                     'Population (2020)': fully_vaccine_pop,
                      'total_vaccinations': people_fvacct
                        }) 

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(30,10))
X = data2['countries']
X_axis = np.arange(len(X))
plt.bar(X_axis -0.0, data2['Population (2020)'], color = 'b', width=0.3)
plt.bar(X_axis +0.0, data2['total_vaccinations'], color ='pink', width=0.40 )

fig.text(0.1,0.9, 'Number total_vaccinations and Number of Population of each Countries', fontsize=23, fontweight='bold', fontfamily='serif',color='black')
fig.text(0.2,0.75, 'Number Population (2020)', fontsize=20, fontweight='bold', fontfamily='serif',color='blue')
fig.text(0.2,0.7, 'Number total_vaccinations', fontsize=20, fontweight='bold', fontfamily='serif',color='pink')
ax.tick_params(length=0)

plt.xticks(X_axis, X, rotation = 90)
plt.grid(axis='x')
plt.grid(axis='y')

#plt.title("Number people of Fully vaccinated and Number of Population of each Countries")
plt.ylabel("x 1,000 Million or 1 Billion " )
plt.xlim([-2,50])

**People of Fully vaccinated Vs Population**

In [ ]:
people_vacc = df.groupby(['country'])['people_fully_vaccinated'].max().sort_values(ascending=False).reset_index()

fully_vaccine_countries =[]
fully_vaccine_pop=[]
people_vacct = []


countries = list(people_vacc['country'])
for pop in countries:
    pop1 = df_pop[df_pop['Country (or dependency)'] == pop]
    pop2 = pop1['Population (2020)'].unique()
    for number in pop2:
        fully_vaccine_pop.append(number)   
        fully_vaccine_countries.append(pop)

data = pd.DataFrame({'countries': fully_vaccine_countries,
                     'Population (2020)': fully_vaccine_pop,
                        })

countries = list(data['countries'])
for pop in countries:
    pop3 = people_vacc[people_vacc['country'] == pop]
    pop4 = pop3['people_fully_vaccinated'].unique()
    for number in pop4:
        people_vacct.append(number)
  
      
data2 = pd.DataFrame({'countries': fully_vaccine_countries,
                     'Population (2020)': fully_vaccine_pop,
                      'fully People vaccinated': people_vacct
                        }) 

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(30,10))
X = data2['countries']
X_axis = np.arange(len(X))
plt.bar(X_axis -0.0, data2['Population (2020)'], color = 'b', width=0.3)
plt.bar(X_axis +0.0, data2['fully People vaccinated'], color ='g', width=0.40 )

fig.text(0.1,0.9, 'Number people of Fully vaccinated and Number of Population of each Countries', fontsize=23, fontweight='bold', fontfamily='serif',color='black')
fig.text(0.2,0.75, 'Number Population (2020)', fontsize=20, fontweight='bold', fontfamily='serif',color='blue')
fig.text(0.2,0.7, 'Number Fully people vaccinated', fontsize=20, fontweight='bold', fontfamily='serif',color='green')
ax.tick_params(length=0)

plt.xticks(X_axis, X, rotation = 90)
plt.grid(axis='x')
plt.grid(axis='y')

#plt.title("Number people of Fully vaccinated and Number of Population of each Countries")
plt.ylabel("x 1,000 Million or 1 Billion " )
plt.xlim([-2,50])

**people vaccinated Vs Population of each Countries**

In [ ]:
people_vacc = df.groupby(['country'])['people_vaccinated'].max().sort_values(ascending=False).reset_index()

fully_vaccine_countries =[]
fully_vaccine_pop=[]
people_vacct = []


countries = list(people_vacc['country'])
for pop in countries:
    pop1 = df_pop[df_pop['Country (or dependency)'] == pop]
    pop2 = pop1['Population (2020)'].unique()
    for number in pop2:
        fully_vaccine_pop.append(number)   
        fully_vaccine_countries.append(pop)
       
data = pd.DataFrame({'countries': fully_vaccine_countries,
                     'Population (2020)': fully_vaccine_pop,
                        })


countries = list(data['countries'])
for pop in countries:
    pop3 = people_vacc[people_vacc['country'] == pop]
    pop4 = pop3['people_vaccinated'].unique()
    for number in pop4:
        people_vacct.append(number)
  
      
data2 = pd.DataFrame({'countries': fully_vaccine_countries,
                     'Population (2020)': fully_vaccine_pop,
                      'People vaccinated': people_vacct
                        }) 

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(30,10))
X = data2['countries']
X_axis = np.arange(len(X))
plt.bar(X_axis -0.0, data2['Population (2020)'], color = 'b', width=0.3)
plt.bar(X_axis +0.0, data2['People vaccinated'], color ='r', width=0.40 )

fig.text(0.1,0.9, 'Number people of Fully vaccinated and Number of Population of each Countries', fontsize=23, fontweight='bold', fontfamily='serif',color='black')
fig.text(0.2,0.75, 'Number Population (2020)', fontsize=20, fontweight='bold', fontfamily='serif',color='blue')
fig.text(0.2,0.7, 'Number people vaccinated', fontsize=20, fontweight='bold', fontfamily='serif',color='red')
ax.tick_params(length=0)

plt.xticks(X_axis, X, rotation = 90)
plt.grid(axis='x')
plt.grid(axis='y')

#plt.title("Number people of Fully vaccinated and Number of Population of each Countries")
plt.ylabel("x 1,000 Million or 1 Billion " )
plt.xlim([-2,50])

**Predict the Success of COVID-19 vaccination in Jordan.**

In [ ]:
jordan_df = df[df['country'] == 'Jordan']

vaccine_type = jordan_df['vaccines']

# Remove Duplicate from vaccines list 
vacc = list(OrderedDict.fromkeys(vaccine_type))

for i in vacc:
    print('Jordan choose vaccines: ',i)

**Current info**

In [ ]:
data = jordan_df[['country','date','people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred']]
data['date']  = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d')  #b=month's name , %m= month's number
     
fig,ax = plt.subplots(1,1,figsize=(25,8))
sns.barplot(x='date', y='people_vaccinated_per_hundred', data = data, color= 'red')
sns.barplot(x='date', y='people_fully_vaccinated_per_hundred', data = data, color= 'green')

fig.text(0.1,0.9, 'COVID-19 Vaccination of Jordan', fontsize=23, fontweight='bold', fontfamily='serif',color='black')
fig.text(0.2,0.75, '%People vaccined', fontsize=20, fontweight='bold', fontfamily='serif',color='red')
fig.text(0.2,0.7, '%People fully vaccinated', fontsize=20, fontweight='bold', fontfamily='serif',color='green')

#plt.grid(axis='x')
plt.grid(axis='y')


#set X-axis
plt.xticks( rotation = 30)
ax.set_xticks(ax.get_xticks()[::7])

plt.ylabel("Percentage " )

**Now prediction**

In [ ]:
# formula of parabola is y =mx^2+c
m = 1/6
c = -10
m1 = 1/9
c1 = -10

y_est = []
y_est1 = []
date = []
y_act = list(data['people_vaccinated_per_hundred'])
y_act1 = list(data['people_fully_vaccinated_per_hundred'])

t1 = dt.date(2021,3,9) #Jordan ahead-start 'people_vaccinated_per_hundred'
x = 0
y = 0
y1 = 0
while y1 < 100:
    if x >= len(list(data['people_vaccinated_per_hundred'])):
        y_act.append(0)
        y_act1.append(0)
            
    y = (m*(x))+c
    y_est.append(y)
    
    y1 = (m1*(x))+c1
    y_est1.append(y1)   
    
    
    t2 = t1+dt.timedelta(days=x)
    date.append(t2)  
    x += 1
        
data_predict= pd.DataFrame({'date': date,
                            'y_est': y_est,
                            'y_est1': y_est1,
                            'y_act': y_act,
                            'y_act1': y_act1
                        
                           })


data_predict['date'] = pd.to_datetime(data_predict['date']).dt.strftime('%Y-%m-%d')  #b=month's name , %m= month's number
fig,ax = plt.subplots(1,1,figsize=(25,8))
sns.barplot(x='date', y='y_act', data = data_predict, color= 'red')
sns.barplot(x='date', y='y_act1', data = data_predict, color= 'green')
sns.lineplot(x='date', y='y_est', data = data_predict, color= 'red')
sns.lineplot(x='date', y='y_est1', data = data_predict, color= 'green')


#set X-axis, y-axis
plt.xticks(rotation = 70)
plt.yticks(np.arange(0,110,20))
plt.ylim(0)
plt.grid(axis='y')
ax.set_xticks(ax.get_xticks()[::10]) # set x-axis 10 days step.
plt.ylabel("Percentage " )

plt.axvline(x=289, color='r', linestyle ='--')
plt.axvline(x=313, color='g', linestyle ='--')


fig.text(0.1,0.9, 'COVID-19 Vaccination of Jordan', fontsize=23, fontweight='bold', fontfamily='serif',color='black')
fig.text(0.2,0.75, '%People vaccined', fontsize=20, fontweight='bold', fontfamily='serif',color='red')
fig.text(0.4,0.75, '-- Est.100% vaccined', fontsize=20, fontweight='bold', fontfamily='serif',color='red')
fig.text(0.2,0.7, '%People fully vaccinated', fontsize=20, fontweight='bold', fontfamily='serif',color='green')
fig.text(0.4,0.7, '-- Est.100% fully vaccinated', fontsize=20, fontweight='bold', fontfamily='serif',color='green')
plt.show()
#print(data_predict.tail(3))